![Nvidia Logo](./images/nvidia.png)  

## Introduction to LLM Agents in LangGraph and LangChain

This notebook provides an introduction to building LLM-based agents using LangGraph and LangChain. It covers the fundamental implementations required for developing agentic workflows. By the end of this notebook, you will have an understanding of the following concepts:

1. [NVIDIA NIM Endpoints](https://build.nvidia.com/explore/discover) – Learn how to integrate and use NVIDIA NIM endpoints for efficient inference.
2. [Tool Calling using `bind_tools`](https://python.langchain.com/docs/modules/agents/tools/custom_tools/) – Understand how to define and bind tools within an agent.
3. [Agents using `create_react_agent`](https://python.langchain.com/docs/modules/agents/) – Explore the creation of ReAct (Reasoning + Acting) agents using Langgraph.

This notebook serves as a introduction to implementing intelligent agents with modular and scalable workflows.


### Set up

In [3]:
import yaml
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

### Accessing Nvidia NIM endpoints

Here is a list of available models, we will be using `meta/llama-3.1-70b-instruct` for this notebook.

In [ ]:
tool_models = [model for model in ChatNVIDIA.get_available_models() if model.supports_tools]
for elem in tool_models:
    print(elem)

We'll set up the LLM through LangChain's ChatNVIDIA functionality, which provides an interface to NVIDIA NIM chat models. It offers connection to both hosted and local NIMs (Mistral, Llama, etc.), tool calling capabilities, streaming functionality, etc. Here is an example on how to implement that

In [5]:
llm =  ChatNVIDIA(
        model= "meta/llama-3.1-70b-instruct",
        temperature=0.2,
        top_p=0.7,
        max_tokens=4096,
)

# Locally-hosted model example
# llm = ChatNVIDIA(base_url="http://3.145.171.211:8000/v1", model_name="meta/llama-3.1-8b-instruct") 

In [6]:
result = llm.invoke("Hi How are you")
print(result.content)

I'm just a computer program, so I don't have feelings, but thanks for asking! How can I assist you today?


### Tool calling with `.bind_tools()`

In LangChain, [tool calling](https://python.langchain.com/docs/concepts/tool_calling/) allows LLMs to invoke external functions, APIs, or utilities dynamically, extending LLM capabilities beyond text generation. We define tools with the `@tool` decorator, and bind them to llm with `.bind_tool()` function. That tells the LLM which tools are available for using. LLM calls these functions with the proper arguments depending on the prompt it receives.

In [7]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b and return result

    Args:
        a: first int
        b: second int
    """
    return a*b

llm_with_tools = llm.bind_tools([multiply])

/home/nvidia/.local/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:591: UserWarning: Model 'nvdev/meta/llama-3.1-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(


LLM intelligently decides where to call a tool or not depending on the prompt.

In [8]:
llm_with_tools.invoke("Hello world!")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 283, 'total_tokens': 307, 'completion_tokens': 24}, 'finish_reason': 'stop', 'model_name': 'nvdev/meta/llama-3.1-70b-instruct'}, id='run-34a9624d-e628-4af5-a5c4-f3ceafc8533a-0', usage_metadata={'input_tokens': 283, 'output_tokens': 24, 'total_tokens': 307}, role='assistant')

In [9]:
result = llm_with_tools.invoke("What is the multiplication of 56 and 64")
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 56, 'b': 64},
  'id': 'chatcmpl-tool-fb0629aaebde42b4ba91e4d36c93a431',
  'type': 'tool_call'}]

Using `.bind_tools()`, LLM tells us which tool to call, with which arguments. But it doesn't actually invoke the tool. To invoke the tool intelligently we use the `.create_react_agent()` prebuilt function from Langgraph.

### Prebuilt ReAcT agent using  `.create_react_agent()`

ReAct (Reasoning + Acting) is an agent architecture, based on this paper [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/pdf/2210.03629), that combines step-by-step reasoning with tool use. LangGraph provides a prebuilt function [create_react_agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) to easily implement this architecture. It uses `.bind_tools()` under the hood to attach tools to the language model.

In [48]:
# Define Tool 1: Square a Number
@tool
def square(n: int) -> int:
    """Returns the square of a number."""
    return n * n

# Define Tool 2: Multiply Two Numbers
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


agent = create_react_agent(llm, [square, multiply])

# Invoke the agent with a query that requires two tool calls
response = agent.invoke({"messages": "What is (5 squared) multiplied by 2?"})



/home/nvidia/.local/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:591: UserWarning: Model 'nvdev/meta/llama-3.1-405b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(


In [49]:
response['messages'][-1].content

'The answer is 50.'

In [55]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is (5 squared) multiplied by 2?
================================== Ai Message ==================================
Tool Calls:
  square (chatcmpl-tool-3fbc1698aad645b7a9b1a38ebfc3c1f4)
 Call ID: chatcmpl-tool-3fbc1698aad645b7a9b1a38ebfc3c1f4
  Args:
    n: 5
================================= Tool Message =================================
Name: square

25
================================== Ai Message ==================================
Tool Calls:
  multiply (chatcmpl-tool-557233293c6c4011a6d54fcfd2633d88)
 Call ID: chatcmpl-tool-557233293c6c4011a6d54fcfd2633d88
  Args:
    a: 25
    b: 2
================================= Tool Message =================================
Name: multiply

50
================================== Ai Message ==================================

The answer is 50.


Now that you are familiar with the basics of building agents, we will explore how to build a 5G network reconfiguration agent. Please refer to [agentic_pipeline-DLI.ipynb](agentic_pipeline-DLI.ipynb) for more details.

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

In [5]:
model = llm
# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)
@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]


class WeatherResponse(BaseModel):
    """Respond to the user in this format."""

    conditions: str = Field(description="Weather conditions")


# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    model,
    tools=tools,
    # specify the schema for the structured output using `response_format` parameter
    response_format=WeatherResponse,
)

/home/nvidia/.local/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:591: UserWarning: Model 'nvdev/meta/llama-3.1-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(


In [6]:
inputs = {"messages": [("user", "What's the weather in NYC?")]}
response = graph.invoke(inputs)

/home/nvidia/.local/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:814: UserWarning: Model 'nvdev/meta/llama-3.1-70b-instruct' is not known to support structured output. Your output may fail at inference time.
  warnings.warn(


In [7]:
response["structured_response"]

WeatherResponse(conditions='Cloudy')